In [1]:
!pip install azure-storage-blob

In [2]:
from azure.storage.blob import BlobServiceClient
import io
import pandas as pd

# Set up the Azure Blob Storage connections
sas_token = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-05-13T13:01:11Z&st=2024-04-13T05:01:11Z&spr=https,http&sig=X362UUEwAfkGDCBr2G0ZOc2SX%2FLg9LOddGvPHZ%2Fff00%3D"

datasets = {
    "admission": {
        "account_url": "https://sadukedatauseprod.blob.core.windows.net",
        "container_name": "mimiciv",
        "blob_name": "mimiciv/hosp/admissions.csv"
    },
    "d_items": {
        "account_url": "https://sadukedatauseprod.blob.core.windows.net",
        "container_name": "mimiciv",
        "blob_name": "mimiciv/icu/d_items.csv"
    },
    "chartevents": {
        "account_url": "https://sadukedatauseprod.blob.core.windows.net",
        "container_name": "mimiciv",
        "blob_name": "mimiciv/icu/chartevents.csv"
    }
    # "MIMIC-III": {
    #     "account_url": "https://sadukedatauseprod.blob.core.windows.net",
    #     "container_name": "mimiciii",
    #     "blob_name": "ADMISSIONS.csv"
    # },
    # "eICU-CRD": {
    #     "account_url": "https://sadukedatauseprod.blob.core.windows.net",
    #     "container_name": "eicucrd",
    #     "blob_name": "patient.csv"
    #}
}

dfs = dict()
for dataset_name, dataset_config in datasets.items():
    print(f"Accessing {dataset_name} dataset:")

    # Create a BlobServiceClient using the account URL and SAS token
    blob_service_client = BlobServiceClient(
        account_url=dataset_config["account_url"],
        credential=sas_token
    )

    # Get a reference to the container
    container_client = blob_service_client.get_container_client(dataset_config["container_name"])

    # List the blobs in the container
    print(f"Blobs in the {dataset_name} container:")
    for blob in container_client.list_blobs():
        print(blob.name)

    # Read data from a specific blob (CSV file)
    blob_client = container_client.get_blob_client(dataset_config["blob_name"])

    # Download the blob content as text
    blob_content = blob_client.download_blob().readall().decode("utf-8")

    # Process the CSV data using pandas
    df = pd.read_csv(io.StringIO(blob_content))
    dfs[dataset_name]=df
    print(f"{dataset_name} DataFrame head:")
    print(df.head())

    print("---")

Accessing admission dataset:
Blobs in the admission container:
mimiciv/hosp/admissions.csv
mimiciv/hosp/d_hcpcs.csv
mimiciv/hosp/d_icd_diagnoses.csv
mimiciv/hosp/d_icd_procedures.csv
mimiciv/hosp/d_labitems.csv
mimiciv/hosp/diagnoses_icd.csv
mimiciv/hosp/drgcodes.csv
mimiciv/hosp/emar.csv
mimiciv/hosp/emar_detail.csv
mimiciv/hosp/hcpcsevents.csv
mimiciv/hosp/labevents.csv
mimiciv/hosp/microbiologyevents.csv
mimiciv/hosp/omr.csv
mimiciv/hosp/patients.csv
mimiciv/hosp/pharmacy.csv
mimiciv/hosp/poe.csv
mimiciv/hosp/poe_detail.csv
mimiciv/hosp/prescriptions.csv
mimiciv/hosp/procedures_icd.csv
mimiciv/hosp/provider.csv
mimiciv/hosp/services.csv
mimiciv/hosp/transfers.csv
mimiciv/icu/caregiver.csv
mimiciv/icu/chartevents.csv
mimiciv/icu/d_items.csv
mimiciv/icu/datetimeevents.csv
mimiciv/icu/icustays.csv
mimiciv/icu/ingredientevents.csv
mimiciv/icu/inputevents.csv
mimiciv/icu/outputevents.csv
mimiciv/icu/procedureevents.csv
admission DataFrame head:
   subject_id   hadm_id            admittim

In [ ]:
dfs['d_items'].head()

In [ ]:
dfs['chartevents'].head()

In [ ]:
import pandas as pd

# Assuming chartevents and d_items are already loaded as pandas DataFrames
chartevents = chartevents.merge(d_items[['itemid', 'label', 'abbreviation', 'linksto', 'category', 'unitname']], on='itemid', how='left')


In [ ]:
-- Define the SQL query
WITH FilteredJoin AS (
  SELECT
    ce.itemid,
    ce.value,  -- Assuming you want to keep some columns from chartevents
    di.label,
    di.abbreviation,
    di.linksto,
    di.category,
    di.unitname
  FROM
    `physionet-data.mimiciv.chartevents` ce
  LEFT JOIN
    `physionet-data.mimiciv.d_items` di
  ON
    ce.itemid = di.itemid
  WHERE
    LOWER(di.label) LIKE '%restraint%'
)

-- Select unique labels from the filtered join
SELECT DISTINCT label
FROM FilteredJoin
ORDER BY label;
